In [ ]:
storage_account_name = "csvtravitfiledata"
# container_name = "novcsvfiles"
container_name = 'onecsvfile'
folder_path = "/"  # Root folder to read all CSV files
sas_token = ""
csv_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net{folder_path}"
print(csv_path)


abfss://onecsvfile@csvtravitfiledata.dfs.core.windows.net/


In [86]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, row_number,when,col,min,max,unix_timestamp,rank, dense_rank,date_format,count,lead,lag,sum,expr,from_utc_timestamp,substring,abs,from_unixtime,to_date,greatest,to_timestamp,lit,concat,split,current_date, concat_ws,regexp_extract,lpad,first, last,date_add,hour
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("MulRead").getOrCreate()
# spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", sas_token)

In [228]:
folder_path = "/Users/sushantshinde/workspaces/trvi/csv_files/"

25/02/26 00:02:06 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1004309 ms exceeds timeout 120000 ms
25/02/26 00:02:06 WARN SparkContext: Killing executors is not supported by current scheduler.
25/02/26 00:18:00 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [88]:
# Read all CSV files in the folder
windowSpec_csv_filename = Window.partitionBy('FileName').orderBy("FileName")
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(folder_path).withColumn("FileName", input_file_name())\
    .withColumn("date_only", to_date(col("_start")))\
    .withColumn("date_id", dense_rank().over(Window.orderBy('date_only')))
    # .withColumn("FileName", regexp_extract(col("FileName"), r"([^/]+)$", 1)) \
    # .withColumn("file_date", substring(col("FileName"), 1, 8)).withColumn("date_id", concat(lit("ID_"), col("file_date")))
    # .withColumn("date_only", to_date(col("_start")))\
    # .withColumn("_time", from_utc_timestamp(col("_time"), "Asia/Kolkata")).withColumn("_stop", from_utc_timestamp(col("_stop"), "Asia/Kolkata")).withColumn("_start", from_utc_timestamp(col("_start"), "Asia/Kolkata"))\
    # 

    # .withColumn("file_date", substring(col("FileName"), 1, 8))\
#    .withColumn("date_id", concat(lit("ID_"), col("file_date")))\
     
    

In [89]:
df_filling = df.withColumn('VatNo',
                           when((col('Report_VAT1_Filling') == 1) &  (col('Report_VAT2_Filling') == 0) ,1)
                           .when((col('Report_VAT1_Filling') == 0) &  (col('Report_VAT2_Filling') == 1) & (col('Report_VAT3_Filling') == 0) & (col('Report_VAT4_Filling') == 0) & (col('Report_VAT5_Filling') == 0) & (col('Report_VAT6_Filling') == 0) & (col('Report_VAT7_Filling') == 0) ,2)
                           .when((col('Report_VAT1_Filling') == 0) &  (col('Report_VAT2_Filling') == 0) & (col('Report_VAT3_Filling') == 1) & (col('Report_VAT4_Filling') == 0) & (col('Report_VAT5_Filling') == 0) & (col('Report_VAT6_Filling') == 0) & (col('Report_VAT7_Filling') == 0),3)
                           .when((col('Report_VAT1_Filling') == 0) &  (col('Report_VAT2_Filling') == 0) & (col('Report_VAT3_Filling') == 0) & (col('Report_VAT4_Filling') == 1) & (col('Report_VAT5_Filling') == 0) & (col('Report_VAT6_Filling') == 0) & (col('Report_VAT7_Filling') == 0),4)
                           .when((col('Report_VAT1_Filling') == 0) &  (col('Report_VAT2_Filling') == 0) & (col('Report_VAT3_Filling') == 0) & (col('Report_VAT4_Filling') == 0) & (col('Report_VAT5_Filling') == 1) & (col('Report_VAT6_Filling') == 0) & (col('Report_VAT7_Filling') == 0),5)
                           .when((col('Report_VAT1_Filling') == 0) &  (col('Report_VAT2_Filling') == 0) & (col('Report_VAT3_Filling') == 0) & (col('Report_VAT4_Filling') == 0) & (col('Report_VAT5_Filling') == 0) & (col('Report_VAT6_Filling') == 1) & (col('Report_VAT7_Filling') == 0),6)
                           .when((col('Report_VAT1_Filling') == 0) &  (col('Report_VAT2_Filling') == 0) & (col('Report_VAT3_Filling') == 0) & (col('Report_VAT4_Filling') == 0) & (col('Report_VAT5_Filling') == 0) & (col('Report_VAT6_Filling') == 0) & (col('Report_VAT7_Filling') == 1),7))\
                .withColumn(
                        "amount",
                        when(col('Report_VAT1_Filling') == 1, col('Report_VAT1_Mqty'))
                        .when(col('Report_VAT2_Filling') == 1, col('Report_VAT2_Mqty'))
                        .when(col('Report_VAT3_Filling') == 1, col('Report_VAT3_Mqty'))
                        .when(col('Report_VAT4_Filling') == 1, col('Report_VAT4_Mqty'))
                        .when(col('Report_VAT5_Filling') == 1, col('Report_VAT5_Mqty'))
                        .when(col('Report_VAT6_Filling') == 1, col('Report_VAT6_Mqty'))
                        .when(col('Report_VAT7_Filling') == 1, col('Report_VAT7_Mqty')))\
            .withColumn(
                        "RPM",
                        when(col('Report_VAT1_Filling') == 1, col('Report_VAT1_RPM'))
                        .when(col('Report_VAT2_Filling') == 1, col('Report_VAT2_RPM'))
                        .when(col('Report_VAT3_Filling') == 1, col('Report_VAT3_RPM'))
                        .when(col('Report_VAT4_Filling') == 1, col('Report_VAT4_RPM'))
                        .when(col('Report_VAT5_Filling') == 1, col('Report_VAT5_RPM'))
                        .when(col('Report_VAT6_Filling') == 1, col('Report_VAT6_RPM'))
                        .when(col('Report_VAT7_Filling') == 1, col('Report_VAT7_RPM'))  
            )\
            .withColumn(
                        "Recipe",
                        when(col('Report_VAT1_Filling') == 1, col('Report_VAT1_Recipe'))
                        .when(col('Report_VAT2_Filling') == 1, col('Report_VAT2_Recipe'))
                        .when(col('Report_VAT3_Filling') == 1, col('Report_VAT3_Recipe'))
                        .when(col('Report_VAT4_Filling') == 1, col('Report_VAT4_Recipe'))
                        .when(col('Report_VAT5_Filling') == 1, col('Report_VAT5_Recipe'))
                        .when(col('Report_VAT6_Filling') == 1, col('Report_VAT6_Recipe'))
                        .when(col('Report_VAT7_Filling') == 1, col('Report_VAT7_Recipe'))  
            )\
            .withColumn(
                    "Temp",
                    when(col('Report_VAT1_Filling') == 1, col('Report_VAT1Temp'))
                    .when(col('Report_VAT2_Filling') == 1, col('Report_VAT2Temp'))
                    .when(col('Report_VAT3_Filling') == 1, col('Report_VAT3Temp'))
                    .when(col('Report_VAT4_Filling') == 1, col('Report_VAT4Temp'))
                    .when(col('Report_VAT5_Filling') == 1, col('Report_VAT5Temp'))
                    .when(col('Report_VAT6_Filling') == 1, col('Report_VAT6Temp'))
                    .when(col('Report_VAT7_Filling') == 1, col('Report_VAT7Temp'))  
                )

In [90]:
df1 = df_filling.groupBy(col('Report_VAT_Seq'),col('VatNo'),'date_id').agg(min(col('_time')).alias('Date'),date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Filling Start Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Filling end time'),max('amount').alias('Milk Quantity in Vat'),min('Temp').alias('Temp of milk in Vat'),max('RPM').alias('Filling_RPM'),max('Recipe').alias('RecipeNo'),min('_start').alias('order time'))\
    .select('Date','VatNo','Report_VAT_Seq','RecipeNo','Filling_RPM','Filling Start Time','Filling end time','Milk Quantity in Vat','Temp of milk in Vat','order time','date_id').orderBy('Filling Start Time').filter(col('VatNo').isNotNull())
# .withColumn("ID", row_number().over(Window.orderBy("order time"))).drop('order time')

In [91]:
df_self = df1.alias('df1').join(df1.alias('df2'),(col('df1.VatNo') == col('df2.VatNo')) & (col('df1.Report_VAT_Seq') == col('df2.Report_VAT_Seq')) & (col('df1.date_id') != col('df2.date_id'))  & (
                         (unix_timestamp(col("df2.order time")) - unix_timestamp(col("df1.order time")) >= 1800) &  # 30 mins = 1800 sec
                         (unix_timestamp(col("df2.order time")) - unix_timestamp(col("df1.order time")) <= 3100)    # 35 mins = 2100 sec
                     ),'left')\
                     .withColumn('Filling_end_time',
                        when(col('df2.Filling end time').isNotNull(),col('df2.Filling end time'))
                        .otherwise(col('df1.Filling end time')) 
                        )\
                     .withColumn('Milk_Quantity_Vat',
                        when(col('df2.Milk Quantity in Vat').isNotNull(),col('df2.Milk Quantity in Vat'))
                        .otherwise(col('df1.Milk Quantity in Vat')) 
                        ).select('df1.*','Filling_end_time','Milk_Quantity_Vat').drop('Filling end time','df1.Milk Quantity in Vat')\
                    .withColumn("row_number", dense_rank().over(Window.orderBy('Filling Start Time'
).partitionBy("VatNo",'Report_VAT_Seq','Filling_end_time'))).filter(col('row_number') == 1).drop('row_number')
# df_self.display()

In [92]:
df_culture = df.withColumn('VatNo',
                           when(col('Report_VAT1_CAddT') == 1,1)
                           .when(col('Report_VAT2_CAddT') == 1,2)
                           .when(col('Report_VAT3_CAddT') == 1,3)
                           .when(col('Report_VAT4_CAddT') == 1,4)
                           .when(col('Report_VAT5_CAddT') == 1,5)
                           .when(col('Report_VAT6_CAddT') == 1,6)
                           .when(col('Report_VAT7_CAddT') == 1,7))\
                .withColumn('Quantity',
                            when(col('Report_VAT1_CAddT') == 1,col('Report_VAT1_CultureQty'))
                           .when(col('Report_VAT2_CAddT') == 1,col('Report_VAT2_CultureQty'))
                           .when(col('Report_VAT3_CAddT') == 1,col('Report_VAT3_CultureQty'))
                           .when(col('Report_VAT4_CAddT') == 1,col('Report_VAT4_CultureQty'))
                           .when(col('Report_VAT5_CAddT') == 1,col('Report_VAT5_CultureQty'))
                           .when(col('Report_VAT6_CAddT') == 1,col('Report_VAT6_CultureQty'))
                           .when(col('Report_VAT7_CAddT') == 1,col('Report_VAT7_CultureQty')))\
                .withColumn('PH',
                            when(col('Report_VAT1_CAddT') == 1,col('Report_VAT1_PH'))
                           .when(col('Report_VAT2_CAddT') == 1,col('Report_VAT2_PH'))
                           .when(col('Report_VAT3_CAddT') == 1,col('Report_VAT3_PH'))
                           .when(col('Report_VAT4_CAddT') == 1,col('Report_VAT4_PH'))
                           .when(col('Report_VAT5_CAddT') == 1,col('Report_VAT5_PH'))
                           .when(col('Report_VAT6_CAddT') == 1,col('Report_VAT6_PH'))
                           .when(col('Report_VAT7_CAddT') == 1,col('Report_VAT7_PH'))
                            )

In [93]:
df_culture1 = df_culture.groupBy(col('Report_VAT_Seq'),col('VatNo'),'date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Culture_Addition_Time'),max(col('Quantity')).alias('Culture_Quantity'),max('PH').alias('Culture_ph'),min('_start').alias('order time')).select('VatNo','Report_VAT_Seq','Culture_Addition_Time','Culture_Quantity','Culture_ph','order time','date_id').orderBy('Report_VAT_Seq').filter(col('VatNo').isNotNull()).drop('order time','date_id')

In [94]:
df_culture1.orderBy('order time').show()

25/02/25 14:53:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+--------------+---------------------+----------------+----------+
|VatNo|Report_VAT_Seq|Culture_Addition_Time|Culture_Quantity|Culture_ph|
+-----+--------------+---------------------+----------------+----------+
|    1|             1|             06:00:22|             200|      6.54|
|    2|             2|             06:33:22|             200|      6.56|
|    3|             3|             07:06:58|             200|       0.0|
|    4|             4|             07:40:30|             200|       0.0|
|    5|             5|             08:13:58|             200|       0.0|
|    6|             6|             08:45:53|             200|       0.0|
|    7|             7|             09:19:24|             200|       0.0|
|    1|             8|             09:52:52|             200|      6.61|
|    2|             9|             10:26:16|             200|      6.58|
|    3|            10|             10:59:47|             200|      6.56|
|    4|            11|             11:33:48|       

In [95]:
df_join = df_self.alias('df1').join(df_culture1.alias('df2'),(col('df1.VatNo') == col('df2.VatNo')) & 
                                (col('df1.Report_VAT_Seq') == col('df2.Report_VAT_Seq')) & 
                                (
                                    (
                                        (unix_timestamp(col("df2.Culture_Addition_Time"), "HH:mm:ss") >= unix_timestamp(col("df1.Filling Start Time"), "HH:mm:ss")) & 
                                        (unix_timestamp(col("df2.Culture_Addition_Time"), "HH:mm:ss") <= unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss"))
                                    ) |
                                    (
                                        (col("df1.Filling Start Time") > col("df1.Filling_end_time")) &  # Handles crossing midnight case
                                        (
                                            (unix_timestamp(col("df2.Culture_Addition_Time"), "HH:mm:ss") >= unix_timestamp(col("df1.Filling Start Time"), "HH:mm:ss")) | 
                                            (unix_timestamp(col("df2.Culture_Addition_Time"), "HH:mm:ss") <= unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss"))
                                        )
                                    )
                                )
                                ,'left').select('df1.*','df2.Culture_Addition_Time','df2.Culture_Quantity','df2.Culture_ph')

In [96]:
df_join.orderBy('order time').show(37)

25/02/25 14:53:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+--------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------+-------+----------------+-----------------+---------------------+----------------+----------+
|                Date|VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|          order time|date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|
+--------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------+-------+----------------+-----------------+---------------------+----------------+----------+
|2024-10-18 05:55:...|    1|             1|       1|        3.0|          05:55:23|               11486|               31.3|2024-10-18 05:31:...|      1|        06:28:22|            11486|             06:00:22|             200|      6.54|
|2024-10-18 06:28:...|    2|             2| 

In [97]:
df_Reent = df.withColumn('VatNo',
                           when(col('Report_VAT1_RAddT') == 1,1)
                           .when(col('Report_VAT2_RAddT') == 1,2)
                           .when(col('Report_VAT3_RAddT') == 1,3)
                           .when(col('Report_VAT4_RAddT') == 1,4)
                           .when(col('Report_VAT5_RAddT') == 1,5)
                           .when(col('Report_VAT6_RAddT') == 1,6)
                           .when(col('Report_VAT7_RAddT') == 1,7))\
            .withColumn('Quantity',
                         when(col('Report_VAT1_RAddT') == 1,col('Report_VAT1_RenetQty'))
                           .when(col('Report_VAT2_RAddT') == 1,col('Report_VAT2_RenetQty'))
                           .when(col('Report_VAT3_RAddT') == 1,col('Report_VAT3_RenetQty'))
                           .when(col('Report_VAT4_RAddT') == 1,col('Report_VAT4_RenetQty'))
                           .when(col('Report_VAT5_RAddT') == 1,col('Report_VAT5_RenetQty'))
                           .when(col('Report_VAT6_RAddT') == 1,col('Report_VAT6_RenetQty'))
                           .when(col('Report_VAT7_RAddT') == 1,col('Report_VAT7_RenetQty'))
                        )

In [98]:
df_Reent1 = df_Reent.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Rennet_Addition_Time'),max('Quantity').alias('Rennet_Quantity'),min('_start').alias('order time')).select('VatNo','Rennet_Addition_Time','Rennet_Quantity','Report_VAT_Seq','order time').orderBy('order time').filter(col('VatNo').isNotNull()).withColumn("row_num", row_number().over(Window.partitionBy("VatNo", "order time").orderBy("Rennet_Addition_Time")))\
    .filter(col('row_num')== 1).drop("row_num").withColumn("ID", row_number().over(Window.orderBy("order time")))

In [99]:
# df_Reent1.display()

In [100]:
df_join2 = df_join.alias('df1').join(df_Reent1.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& (abs(unix_timestamp(col("df2.Rennet_Addition_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")) <= 1800),'left')\
    .select('df1.*','Rennet_Addition_Time','Rennet_Quantity')

In [101]:
df_join2.orderBy('order time').show()

25/02/25 14:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+--------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+
|                Date|VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|          order time|date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|
+--------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+
|2024-10-18 05:55:...|    1|             1|       1|        3.0|          05:55:23|               11486|               31.3|2024-10-18 05:31:...|      1|        06:28:22|  

In [102]:
windowSpec1 = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(abs(unix_timestamp(col("df2.Rennet_Addition_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")))

df_join2 =df_join2.withColumn("row_num", row_number().over(windowSpec1)).filter(col("row_num") == 1).drop("row_num")


# df_join2.orderBy('order time').display()

In [103]:
df_cogt = df.withColumn('VatNo',
                        when((col('Report_VAT1_CogT') == 1) & ((col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),1)
                           .when((col('Report_VAT2_CogT') == 1) & ((col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0)),2)
                           .when((col('Report_VAT3_CogT') == 1 ) & ((col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0)),3)
                           .when((col('Report_VAT4_CogT') == 1) & ((col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,4)
                           .when((col('Report_VAT5_CogT') == 1) & ((col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,5)
                           .when((col('Report_VAT6_CogT') == 1) & ((col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,6)
                           .when((col('Report_VAT7_CogT') == 1) & ((col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,7))

df_cogt1 = df_cogt.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Coagulation_Start_Time'),
               date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Coagulation_Stop_Time'),min('_start').alias('order time'),min('FileName').alias('FileName'))\
   .select('VatNo','Coagulation_Start_Time','Coagulation_Stop_Time','Report_VAT_Seq','order time','date_id','FileName').filter(col('VatNo').isNotNull()).withColumn("join_time", substring(col("Coagulation_Stop_Time"), 1, 5)).withColumn("ID", row_number().over(Window.orderBy("order time")))

In [104]:
df_cogt1.orderBy('order time').show(31,truncate=False)

25/02/25 14:53:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+----------------------+---------------------+--------------+--------------------------+-------+---------------------------------------------------------------------------+---------+---+
|VatNo|Coagulation_Start_Time|Coagulation_Stop_Time|Report_VAT_Seq|order time                |date_id|FileName                                                                   |join_time|ID |
+-----+----------------------+---------------------+--------------+--------------------------+-------+---------------------------------------------------------------------------+---------+---+
|1    |06:35:12              |07:01:57             |2             |2024-10-18 06:31:01.285304|1      |file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018013105.v0.csv|07:01    |1  |
|2    |07:08:06              |07:35:28             |3             |2024-10-18 07:01:01.289936|1      |file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018020106.v0.csv|07:35    |2  |
|3    |07:41:36              |08:08

In [105]:
# df_test = df.withColumn('VatNo',
#                         when((col('Report_VAT1_CogT') == 1) & ((col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),1)
#                            .when((col('Report_VAT2_CogT') == 1) & ((col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0)),2)
#                            .when((col('Report_VAT3_CogT') == 1 ) & ((col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0)),3)
#                            .when((col('Report_VAT4_CogT') == 1) & ((col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0) | (col('Report_VAT5_Filling') == 0)) ,4)
#                            .when((col('Report_VAT5_CogT') == 1) & ((col('Report_VAT4_CogT') == 1) |(col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0))  ,5)
#                            .when((col('Report_VAT6_CogT') == 1) & ((col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,6)
#                            .when((col('Report_VAT7_CogT') == 1) & ((col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,7))
# df_cogt2 = df_test.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Coagulation_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Coagulation_Stop_Time'),min('_start').alias('order time'),min('FileName').alias('FileName'))\
#    .select('VatNo','Coagulation_Start_Time','Coagulation_Stop_Time','Report_VAT_Seq','order time','date_id','FileName').filter(col('VatNo').isNotNull())\
#    .withColumn("join_time", substring(col("Coagulation_Start_Time"), 1, 5))

In [106]:
df_test1 = df.withColumn('VatNo',
                        when((col('Report_VAT1_CogT') == 1) ,1)
                           .when((col('Report_VAT2_CogT') == 1) ,2)
                           .when((col('Report_VAT3_CogT') == 1 ) ,3)
                           .when((col('Report_VAT4_CogT') == 1)  ,4)
                           .when((col('Report_VAT5_CogT') == 1)  ,5)
                           .when((col('Report_VAT6_CogT') == 1)  ,6)
                           .when((col('Report_VAT7_CogT') == 1) ,7))
df_cogt_testing = df_test1.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Coagulation_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Coagulation_Stop_Time'),min('_start').alias('order time'),min('FileName').alias('FileName'))\
   .select('VatNo','Coagulation_Start_Time','Coagulation_Stop_Time','Report_VAT_Seq','order time','date_id','FileName').filter(col('VatNo').isNotNull())\
   .withColumn("join_time", substring(col("Coagulation_Start_Time"), 1, 5)).withColumn(
    "Cooking_Start_Time_Cleaned", date_format("Coagulation_Start_Time", "HH:mm")
).withColumn(
    "Cooking_Stop_Time_Cleaned", date_format("Coagulation_Stop_Time", "HH:mm")
)


In [107]:
# df_cogt_testing.count()

In [108]:
# df_cogt_testing.orderBy('order time').show(df_cogt_testing.count(),truncate=False)

In [109]:

df_cogt2 = df_cogt_testing.alias('df1').join(
    df_cogt_testing.alias('df2'),
    (col('df1.VatNo') == col('df2.VatNo')) & 
    (date_format(col("df1.Coagulation_Stop_Time"), "HH:mm") == date_format(col("df2.Coagulation_Start_Time"), "HH:mm")),
    'left'
).withColumn('Coagulation Stop Time',
                        when(col('df2.Coagulation_Stop_Time').isNotNull(),col('df2.Coagulation_Stop_Time'))
                        .otherwise(col('df1.Coagulation_Stop_Time')) 
).select('df1.VatNo','df1.Coagulation_Start_Time','df1.Report_VAT_Seq',col('Coagulation Stop Time').alias('Coagulation_Stop_Time'))\
.withColumn("row_number", dense_rank().over(Window.orderBy('Coagulation_Start_Time').partitionBy("VatNo",'Coagulation_Stop_Time'))).filter(col('row_number') == 1).drop('row_number')

df_cogt2.orderBy('df1.order time').show(df_cogt2.count(),truncate=False)

25/02/25 14:53:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:53:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+----------------------+--------------+---------------------+
|VatNo|Coagulation_Start_Time|Report_VAT_Seq|Coagulation_Stop_Time|
+-----+----------------------+--------------+---------------------+
|1    |06:35:12              |2             |07:09:38             |
|2    |07:09:39              |3             |07:42:56             |
|3    |07:42:57              |4             |08:16:02             |
|4    |08:16:24              |5             |08:50:57             |
|5    |08:50:58              |6             |09:21:37             |
|6    |09:22:38              |7             |09:57:15             |
|7    |09:57:16              |8             |10:27:29             |
|1    |10:27:30              |9             |11:02:20             |
|2    |11:02:21              |10            |11:35:20             |
|3    |11:35:21              |11            |12:09:44             |
|4    |12:09:45              |12            |12:43:40             |
|5    |12:43:41              |13            |13:

In [161]:
df_coagt_self = df_cogt2.alias('df1').join(df_cogt1.alias('df2'),(col('df1.VatNo') == col('df2.VatNo')) & (
    (
                                    (
                                        (unix_timestamp(col("df2.Coagulation_Stop_Time"), "HH:mm:ss") >= unix_timestamp(col("df1.Coagulation_Start_Time"), "HH:mm:ss")) & 
                                        (unix_timestamp(col("df2.Coagulation_Stop_Time"), "HH:mm:ss") <= unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss"))
                                    ) |
                                    (
                                        (col("df1.Coagulation_Start_Time") > col("df1.Coagulation_Stop_Time")) &  # Handles crossing midnight case
                                        (
                                            (unix_timestamp(col("df2.Coagulation_Stop_Time"), "HH:mm:ss") >= unix_timestamp(col("df1.Coagulation_Start_Time"), "HH:mm:ss")) | 
                                            (unix_timestamp(col("df2.Coagulation_Stop_Time"), "HH:mm:ss") <= unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss"))
                                        )
                                    )
                                )
),'left')\
.withColumn('Coagulation Start Time',
                        when(col('df2.Coagulation_Start_Time').isNotNull(),col('df2.Coagulation_Start_Time'))
                        .otherwise(col('df1.Coagulation_Start_Time')))\
.select('df1.VatNo','df1.Coagulation_Stop_Time','df1.Report_VAT_Seq',col('Coagulation Start Time').alias('Coagulation_Start_Time'),'order time')

In [162]:
df_coagt_self.orderBy('order time').show(df_coagt_self.count(),truncate=False)

25/02/25 15:02:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+---------------------+--------------+----------------------+--------------------------+
|VatNo|Coagulation_Stop_Time|Report_VAT_Seq|Coagulation_Start_Time|order time                |
+-----+---------------------+--------------+----------------------+--------------------------+
|3    |00:02:55             |33            |00:01:01              |NULL                      |
|5    |21:15:24             |28            |21:08:00              |NULL                      |
|1    |07:09:38             |2             |06:35:12              |2024-10-18 06:31:01.285304|
|2    |07:42:56             |3             |07:08:06              |2024-10-18 07:01:01.289936|
|3    |08:16:02             |4             |07:41:36              |2024-10-18 07:31:01.291004|
|4    |08:50:57             |5             |08:16:24              |2024-10-18 08:01:01.286304|
|5    |09:21:37             |6             |08:47:11              |2024-10-18 08:31:01.286795|
|6    |09:57:15             |7             |09:22:

25/02/25 15:02:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:02:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

In [112]:
# df_cogt2.orderBy('order time').show(31,truncate=False)

In [113]:
# df_coagt = df_cogt1.alias('df1').join(df_cogt2.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& (abs(unix_timestamp(col("df1.join_time"),"HH:mm") - unix_timestamp(col("df2.join_time"),"HH:mm")) <= 180),'left')\
#     .orderBy('df1.Report_VAT_Seq')\
#     .select('df1.*',col('df2.Coagulation_Stop_Time').alias('stop time'))


In [114]:
# df_coagt.orderBy('order time').show(31,truncate=False)

In [115]:
# df_coagt2 = df_coagt.withColumn('Coagulation Stop Time',
#                              when(col('stop time').isNull(),col('Coagulation_Stop_Time'))
#                              .otherwise(col('stop time')) 
#                                 ).withColumn("row_num", row_number().over(Window.partitionBy("VatNo", "Coagulation Stop Time").orderBy("Coagulation_Stop_Time"))).filter(col('row_num')== 1).drop("row_num","join_time","stop time",'stop time','Coagulation_Stop_Time','ID')
#                                 #  .withColumn("ID", row_number().over(Window.orderBy('Report_VAT_Seq',"order time")))

In [116]:
# df_coagt_self =df_coagt2.alias('df1').join(df_coagt2.alias('df2'),(col('df1.VatNo') == col('df2.VatNo')) & (col('df1.Report_VAT_Seq') == col('df2.Report_VAT_Seq')) & (col('df1.date_id') != col('df2.date_id'))  & (
#                          (unix_timestamp(col("df2.order time")) - unix_timestamp(col("df1.order time")) >= 1800) &  # 30 mins = 1800 sec
#                          (unix_timestamp(col("df2.order time")) - unix_timestamp(col("df1.order time")) <= 2100)    # 35 mins = 2100 sec
#                      ),'left')\
#                      .withColumn('Coagulation_Stop_Time',
#                         when(col('df2.Coagulation Stop Time').isNotNull(),col('df2.Coagulation Stop Time'))
#                         .otherwise(col('df1.Coagulation Stop Time')) 
#                         )\
#                     .select('df1.*','Coagulation_Stop_Time').drop('Coagulation Stop Time')\
#                     .withColumn("row_number", dense_rank().over(Window.orderBy('order time'
# ).partitionBy("VatNo",'Report_VAT_Seq','Coagulation_Stop_Time')))\
# .filter(col('row_number') == 1).drop('row_number')

In [160]:
df_coagt_self.orderBy('order time').show(37,truncate=False)

25/02/25 14:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:59:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+---------------------+--------------+----------------------+
|VatNo|Coagulation_Stop_Time|Report_VAT_Seq|Coagulation_Start_Time|
+-----+---------------------+--------------+----------------------+
|3    |00:02:55             |33            |00:01:01              |
|5    |21:15:24             |28            |21:08:00              |
|1    |07:09:38             |2             |06:35:12              |
|2    |07:42:56             |3             |07:08:06              |
|3    |08:16:02             |4             |07:41:36              |
|4    |08:50:57             |5             |08:16:24              |
|5    |09:21:37             |6             |08:47:11              |
|6    |09:57:15             |7             |09:22:38              |
|7    |10:27:29             |8             |09:54:08              |
|1    |11:02:20             |9             |10:27:30              |
|2    |11:35:20             |10            |11:00:48              |
|3    |12:09:44             |11            |11:3

In [198]:
windowSpec3 = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(abs(unix_timestamp(col("df2.Coagulation_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")))

df_coagt_join =  df_join2.alias('df1').join(df_coagt_self.alias('df2'),
                (col('df1.VatNo') == col('df2.VatNo') )& 
                (abs(unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`df2`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df2`.`Coagulation_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ) - unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`df1`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df1`.`Filling_end_time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    )) <= 2800),'left')\
    .select('df1.*','Coagulation_Start_Time','Coagulation_Stop_Time',col('df2.order time').alias('coagt order time')).withColumn("row_num", row_number().over(windowSpec3)).filter(col("row_num") == 1).drop("row_num")

In [199]:
df_coagt_join.orderBy('order time').show(df_coagt_join.count(),truncate=False)

# abs(unix_timestamp(col("df2.Coagulation_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")
# abs(
#                     unix_timestamp(
#                         to_timestamp(expr("CONCAT(date_format(`df2`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df2`.`Cooking_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
#                     ) - 
#                     unix_timestamp(
#                         to_timestamp(expr("CONCAT(date_format(`df1`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df1`.`Coagulation_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
#                     )
#                 )

25/02/25 15:37:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:37:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:37:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:37:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:37:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:37:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+----------------------+---------------------+--------------------------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|order time                |date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|Coagulation_Start_Time|Coagulation_Stop_Time|coagt order time          |
+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+--

In [200]:
df_cutting = df.withColumn('VatNo',
                           when((col('Report_VAT1_CutT') == 1) &  ( (col('Report_VAT7_CutT') == 0) | (col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),1)
                           .when((col('Report_VAT2_CutT') == 1) &  ( (col('Report_VAT1_CutT') == 0) | (col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),2)
                           .when((col('Report_VAT3_CutT') == 1) & ( (col('Report_VAT2_CutT') == 0) | (col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),3)
                           .when((col('Report_VAT4_CutT') == 1) & ((col('Report_VAT3_CutT') == 0) |(col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),4)
                           .when((col('Report_VAT5_CutT') == 1) & ( (col('Report_VAT4_CutT') == 0) |(col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),5)
                           .when((col('Report_VAT6_CutT') == 1) & ( (col('Report_VAT5_CutT') == 0) | (col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),6)
                           .when((col('Report_VAT7_CutT') == 1)  & ( (col('Report_VAT6_CutT') == 0) |(col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),7))\
                .withColumn("RPM",when(col('Report_VAT1_CutT') == 1,col('Report_VAT1_RPM'))
                           .when(col('Report_VAT2_CutT') == 1,col('Report_VAT2_RPM'))
                           .when(col('Report_VAT3_CutT') == 1,col('Report_VAT3_RPM'))
                           .when(col('Report_VAT4_CutT') == 1,col('Report_VAT4_RPM'))
                           .when(col('Report_VAT5_CutT') == 1,col('Report_VAT5_RPM'))
                           .when(col('Report_VAT6_CutT') == 1,col('Report_VAT6_RPM'))
                           .when(col('Report_VAT7_CutT') == 1,col('Report_VAT7_RPM')))

In [201]:
df_cutting1 = df_cutting.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Cutting_Start_Time'),max('RPM').alias('Cutting_RPM'),min('_start').alias('order time'),min('FileName').alias('FileName'))\
    .select('VatNo','Cutting_Start_Time','Cutting_RPM','Report_VAT_Seq','order time','FileName','date_id').filter(col('VatNo').isNotNull())
# .withColumn("Prev_VatNo", lag("VatNo").over(Window.orderBy("order time"))).filter(
#     (col("Report_VAT_Seq") != 0) | (col("VatNo") == col("Prev_VatNo") + 1)).drop('Prev_VatNo').withColumn("ID", row_number().over(Window.orderBy('Report_VAT_Seq',"order time")))

In [203]:
df_cutting1.orderBy('order time').show(df_cutting1.count(),truncate=False)
df_cutting1.count()

25/02/25 15:38:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+------------------+-----------+--------------+--------------------------+---------------------------------------------------------------------------+-------+
|VatNo|Cutting_Start_Time|Cutting_RPM|Report_VAT_Seq|order time                |FileName                                                                   |date_id|
+-----+------------------+-----------+--------------+--------------------------+---------------------------------------------------------------------------+-------+
|1    |07:10:41          |7.0        |3             |2024-10-18 07:01:01.289936|file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018020106.v0.csv|1      |
|2    |07:43:59          |7.0        |4             |2024-10-18 07:31:01.291004|file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018023106.v0.csv|1      |
|3    |08:17:05          |7.0        |5             |2024-10-18 08:01:01.286304|file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018030105.v0.csv|1      |
|4    |08:

25/02/25 15:38:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


69

In [202]:
windowSpec = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(abs(unix_timestamp(col("df2.Cutting_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss")))

df_cuuting_join =  df_coagt_join.alias('df1').join(df_cutting1.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& (abs(unix_timestamp(col("df2.Cutting_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss")) <= 1800),'left').select('df1.*','Cutting_Start_Time','Cutting_RPM')\
.withColumn("row_num", row_number().over(windowSpec)).filter(col("row_num") == 1).drop("row_num")

In [169]:
df_cuuting_join.orderBy('order time').show(df_cuuting_join.count(),truncate=False)

25/02/25 15:10:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:10:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:10:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:10:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:10:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:10:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+----------------------+---------------------+------------------+-----------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|order time                |date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|Coagulation_Start_Time|Coagulation_Stop_Time|Cutting_Start_Time|Cutting_RPM|
+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+----------

In [204]:
df_inter_cutt = df.withColumn('VatNo',
                            when((col('Report_VAT1_IntT') == 1) & ( (col('Report_VAT7_IntT') == 0) | (col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),1)
                           .when((col('Report_VAT2_IntT') == 1) & ( (col('Report_VAT1_IntT') == 0) | (col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),2)
                           .when((col('Report_VAT3_IntT') == 1) & ((col('Report_VAT2_IntT') == 0) | (col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),3)
                           .when((col('Report_VAT4_IntT') == 1) & ((col('Report_VAT3_IntT') == 0) | (col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),4)
                           .when((col('Report_VAT5_IntT') == 1) & ((col('Report_VAT4_IntT') == 0) | (col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),5)
                           .when((col('Report_VAT6_IntT') == 1) & ( (col('Report_VAT5_IntT') == 0) |(col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),6)
                           .when((col('Report_VAT7_IntT') == 1) & ((col('Report_VAT6_IntT') == 0) | (col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),7))\
                .withColumn("RPM",when(col('Report_VAT1_IntT') == 1,col('Report_VAT1_RPM'))
                           .when(col('Report_VAT2_IntT') == 1,col('Report_VAT2_RPM'))
                           .when(col('Report_VAT3_IntT') == 1,col('Report_VAT3_RPM'))
                           .when(col('Report_VAT4_IntT') == 1,col('Report_VAT4_RPM'))
                           .when(col('Report_VAT5_IntT') == 1,col('Report_VAT5_RPM'))
                           .when(col('Report_VAT6_IntT') == 1,col('Report_VAT6_RPM'))
                           .when(col('Report_VAT7_IntT') == 1,col('Report_VAT7_RPM')))

In [205]:
df_inter_cutt1 = df_inter_cutt.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Intermittent_Cutting_Start_Time'),min('_start').alias('order time'),max('RPM').alias('Intermittent_Cutting_RPM')).select('VatNo','Intermittent_Cutting_Start_Time','Intermittent_Cutting_RPM','Report_VAT_Seq','order time').filter(col('VatNo').isNotNull())\
# .filter((col("Report_VAT_Seq") == 0) & (col("VatNo") == min_vatno) | (col("Report_VAT_Seq") != 0) )\
# .withColumn("ID", row_number().over(Window.orderBy('Report_VAT_Seq',"order time")))

In [206]:
windowSpec_inter = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(abs(unix_timestamp(col("df2.Intermittent_Cutting_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss")))

df_inter_cutt_join =  df_cuuting_join.alias('df1').join(df_inter_cutt1.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& (abs(unix_timestamp(col("df2.Intermittent_Cutting_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss")) <= 1800),'left').select('df1.*','Intermittent_Cutting_Start_Time','Intermittent_Cutting_RPM')\
.withColumn("row_num", row_number().over(windowSpec_inter)).filter(col("row_num") == 1).drop("row_num")

In [207]:
df_inter_cutt_join.orderBy('order time').show(df_inter_cutt_join.count(),truncate=False)

25/02/25 15:38:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:38:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+----------------------+---------------------+--------------------------+------------------+-----------+-------------------------------+------------------------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|order time                |date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|Coagulation_Start_Time|Coagulation_Stop_Time|coagt order time          |Cutting_Start_Time|Cutting_RPM|Intermittent_Cutting_Start_Time|Intermittent_Cutting_RPM|
+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+---

In [208]:
df_cooking = df.withColumn('VatNo',
                           when((col('Report_VAT1_CookT') == 1)  & (col('Report_VAT1_Filling') == 0) & (col('Report_VAT1_CAddT') == 0) & (col('Report_VAT1_RAddT') == 0) & (col('Report_VAT1_CutT') == 0) & (col('Report_VAT1_IntT') == 0),1)
                           .when((col('Report_VAT2_CookT') == 1) & (col('Report_VAT2_Filling') == 0) & (col('Report_VAT2_CAddT') == 0) & (col('Report_VAT2_RAddT') == 0) & (col('Report_VAT2_CutT') == 0) & (col('Report_VAT2_IntT') == 0),2)
                           .when((col('Report_VAT3_CookT') == 1) & (col('Report_VAT3_Filling') == 0) & (col('Report_VAT3_CAddT') == 0) & (col('Report_VAT3_RAddT') == 0) & (col('Report_VAT3_CutT') == 0) & (col('Report_VAT3_IntT') == 0),3)
                           .when((col('Report_VAT4_CookT') == 1)  & (col('Report_VAT4_Filling') == 0) & (col('Report_VAT4_CAddT') == 0) & (col('Report_VAT4_RAddT') == 0) & (col('Report_VAT4_CutT') == 0) & (col('Report_VAT4_IntT') == 0),4)
                           .when((col('Report_VAT5_CookT') == 1)  & (col('Report_VAT5_Filling') == 0) & (col('Report_VAT5_CAddT') == 0) & (col('Report_VAT5_RAddT') == 0) & (col('Report_VAT5_CutT') == 0) & (col('Report_VAT5_IntT') == 0),5)
                           .when((col('Report_VAT6_CookT') == 1)  & (col('Report_VAT6_Filling') == 0) & (col('Report_VAT6_CAddT') == 0) & (col('Report_VAT6_RAddT') == 0) & (col('Report_VAT6_CutT') == 0) & (col('Report_VAT6_IntT') == 0),6)
                           .when((col('Report_VAT7_CookT') == 1)  & (col('Report_VAT7_Filling') == 0) & (col('Report_VAT7_CAddT') == 0) & (col('Report_VAT7_RAddT') == 0) & (col('Report_VAT7_CutT') == 0) & (col('Report_VAT7_IntT') == 0),7))\
                .withColumn("Temp",when(col('Report_VAT1_CookT') == 1,col('Report_VAT1Temp'))
                           .when(col('Report_VAT2_CookT') == 1,col('Report_VAT2Temp'))
                           .when(col('Report_VAT3_CookT') == 1,col('Report_VAT3Temp'))
                           .when(col('Report_VAT4_CookT') == 1,col('Report_VAT4Temp'))
                           .when(col('Report_VAT5_CookT') == 1,col('Report_VAT5Temp'))
                           .when(col('Report_VAT6_CookT') == 1,col('Report_VAT6Temp'))
                           .when(col('Report_VAT7_CookT') == 1,col('Report_VAT7Temp')))

In [209]:
df_cooking1 = df_cooking.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Cooking_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Cooking_Stop_Time'),min('_start').alias('order time'),max('Temp').alias('Temp cookin'),min('FileName').alias('FileName'))\
    .select('VatNo','Cooking_Start_Time','Cooking_Stop_Time','Report_VAT_Seq','order time','date_id','Temp cookin','FileName').filter(col('VatNo').isNotNull()).orderBy('Cooking_Start_Time','Report_VAT_Seq',)\
.withColumn("Date_Hour", date_format(col("order time"), "yyyy-MM-dd HH"))
# \
# .withColumn(
#     "Group_Flag",
#     when((col("Prev_Stop_Time").isNotNull()) & (col("Cooking_Start_Time") > col("Prev_Stop_Time")), 1).otherwise(0)
# )

In [176]:
df_cooking1.orderBy('order time').show(df_cooking1.count(),truncate=False)

25/02/25 15:12:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:12:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:12:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:12:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:12:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:12:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+------------------+-----------------+--------------+--------------------------+-------+-----------+---------------------------------------------------------------------------+-------------+
|VatNo|Cooking_Start_Time|Cooking_Stop_Time|Report_VAT_Seq|order time                |date_id|Temp cookin|FileName                                                                   |Date_Hour    |
+-----+------------------+-----------------+--------------+--------------------------+-------+-----------+---------------------------------------------------------------------------+-------------+
|1    |06:28:23          |06:29:56         |2             |2024-10-18 06:01:01.284471|1      |31.5       |file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018010106.v0.csv|2024-10-18 06|
|1    |07:20:15          |07:35:29         |3             |2024-10-18 07:01:01.289936|1      |34.6       |file:///Users/sushantshinde/workspaces/trvi/csv_files/20241018020106.v0.csv|2024-10-18 07|
|2    |07:01:58

In [210]:
df_group_cook = df_cooking1.groupBy('VatNo','Date_Hour').agg(min('Cooking_Start_Time').alias('Cooking_Start_Time'),max('Cooking_Stop_Time').alias('Cooking_Stop_Time'),max('Temp cookin').alias('max temp'))


df_group_cook.orderBy('Date_Hour').show(df_group_cook.count(),truncate=False)


25/02/25 15:39:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+-------------+------------------+-----------------+--------+
|VatNo|Date_Hour    |Cooking_Start_Time|Cooking_Stop_Time|max temp|
+-----+-------------+------------------+-----------------+--------+
|1    |2024-10-18 06|06:28:23          |06:29:56         |31.5    |
|1    |2024-10-18 07|07:20:15          |07:54:38         |38.9    |
|2    |2024-10-18 07|07:01:58          |08:08:54         |35.0    |
|2    |2024-10-18 08|08:08:55          |08:28:08         |38.9    |
|3    |2024-10-18 08|08:28:09          |09:05:38         |38.9    |
|4    |2024-10-18 09|09:05:39          |09:40:45         |38.9    |
|5    |2024-10-18 09|09:40:46          |10:06:46         |39.1    |
|1    |2024-10-18 10|10:21:12          |10:22:02         |32.0    |
|2    |2024-10-18 10|10:54:41          |10:55:32         |32.4    |
|6    |2024-10-18 10|10:07:59          |10:42:18         |39.1    |
|7    |2024-10-18 10|10:42:19          |11:13:57         |38.8    |
|1    |2024-10-18 11|11:13:58          |11:48:34

In [178]:

df_result = df_group_cook.alias('df1').join(df_group_cook.alias('df2'), (col('df1.VatNo') == col('df2.VatNo')) &  (date_format(col("df1.Cooking_Start_Time"), "HH:mm") == date_format(col("df2.Cooking_Stop_Time"), "HH:mm")),'inner').select('df1.VatNo','df1.Cooking_Start_Time','df1.Cooking_Stop_Time','df2.max temp')

# df_result.display()

In [211]:
df_join_cooking_df =  df_group_cook.alias('df1').join(df_result.alias('df2'), (col('df1.VatNo') == col('df2.VatNo')) &  (date_format(col("df1.Cooking_Stop_Time"), "HH:mm") == date_format(col("df2.Cooking_Start_Time"), "HH:mm")),'left').select('df1.VatNo','df1.Cooking_Start_Time','df1.max temp','df1.Date_Hour',
when(col("df2.Cooking_Stop_Time").isNotNull(), col("df2.Cooking_Stop_Time")).otherwise(col("df1.Cooking_Stop_Time")).alias('Cooking_Stop_Time')).withColumn("Start_Timestamp", unix_timestamp(concat_ws(" ", date_format(col("df1.Date_Hour"), "yyyy-MM-dd"), col("Cooking_Stop_Time")), "yyyy-MM-dd HH:mm:ss"))

In [212]:
df_join_cooking_df.orderBy('Date_Hour').show(df_join_cooking_df.count(),truncate=False)

df_join_cooking_df.count()

25/02/25 15:39:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+------------------+--------+-------------+-----------------+---------------+
|VatNo|Cooking_Start_Time|max temp|Date_Hour    |Cooking_Stop_Time|Start_Timestamp|
+-----+------------------+--------+-------------+-----------------+---------------+
|1    |06:28:23          |31.5    |2024-10-18 06|06:29:56         |1729213196     |
|1    |07:20:15          |38.9    |2024-10-18 07|07:54:38         |1729218278     |
|2    |07:01:58          |35.0    |2024-10-18 07|08:28:08         |1729220288     |
|2    |08:08:55          |38.9    |2024-10-18 08|08:28:08         |1729220288     |
|3    |08:28:09          |38.9    |2024-10-18 08|09:05:38         |1729222538     |
|4    |09:05:39          |38.9    |2024-10-18 09|09:40:45         |1729224645     |
|5    |09:40:46          |39.1    |2024-10-18 09|10:06:46         |1729226206     |
|1    |10:21:12          |32.0    |2024-10-18 10|10:22:02         |1729227122     |
|2    |10:54:41          |32.4    |2024-10-18 10|10:55:32         |172922913

25/02/25 15:39:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

57

In [213]:
df_cooking_testing = df.withColumn('VatNo',
                           when((col('Report_VAT1_CookT') == 1) & ( (col('Report_VAT7_CookT') == 0) | (col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),1)
                           .when((col('Report_VAT2_CookT') == 1) & ( (col('Report_VAT1_CookT') == 0) | (col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0)),2)
                           .when((col('Report_VAT3_CookT') == 1 ) & ((col('Report_VAT2_CookT') == 0) | (col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0)),3)
                           .when((col('Report_VAT4_CookT') == 1) & ((col('Report_VAT3_CookT') == 0) | (col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,4)
                           .when((col('Report_VAT5_CookT') == 1) & ((col('Report_VAT4_CookT') == 0) | (col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,5)
                           .when((col('Report_VAT6_CookT') == 1) & ((col('Report_VAT5_CookT') == 0) | (col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,6)
                           .when((col('Report_VAT7_CookT') == 1) & ((col('Report_VAT6_CookT') == 0) | (col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,7))\
                        .withColumn("RPM",when(col('Report_VAT1_CookT') == 1,col('Report_VAT1_RPM'))
                           .when(col('Report_VAT2_CookT') == 1,col('Report_VAT2_RPM'))
                           .when(col('Report_VAT3_CookT') == 1,col('Report_VAT3_RPM'))
                           .when(col('Report_VAT4_CookT') == 1,col('Report_VAT4_RPM'))
                           .when(col('Report_VAT5_CookT') == 1,col('Report_VAT5_RPM'))
                           .when(col('Report_VAT6_CookT') == 1,col('Report_VAT6_RPM'))
                           .when(col('Report_VAT7_CookT') == 1,col('Report_VAT7_RPM')))\
                .withColumn("PH",when(col('Report_VAT1_CookT') == 1,col('Report_VAT1_PH'))
                           .when(col('Report_VAT2_CookT') == 1,col('Report_VAT2_PH'))
                           .when(col('Report_VAT3_CookT') == 1,col('Report_VAT3_PH'))
                           .when(col('Report_VAT4_CookT') == 1,col('Report_VAT4_PH'))
                           .when(col('Report_VAT5_CookT') == 1,col('Report_VAT5_PH'))
                           .when(col('Report_VAT6_CookT') == 1,col('Report_VAT6_PH'))
                           .when(col('Report_VAT7_CookT') == 1,col('Report_VAT7_PH')))\
                .withColumn("Temp",when(col('Report_VAT1_CookT') == 1,col('Report_VAT1Temp'))
                           .when(col('Report_VAT2_CookT') == 1,col('Report_VAT2Temp'))
                           .when(col('Report_VAT3_CookT') == 1,col('Report_VAT3Temp'))
                           .when(col('Report_VAT4_CookT') == 1,col('Report_VAT4Temp'))
                           .when(col('Report_VAT5_CookT') == 1,col('Report_VAT5Temp'))
                           .when(col('Report_VAT6_CookT') == 1,col('Report_VAT6Temp'))
                           .when(col('Report_VAT7_CookT') == 1,col('Report_VAT7Temp')))

In [214]:
df_cooking_testing1 = df_cooking_testing.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Cooking_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Cooking_Stop_Time'),min('_start').alias('order time'),max('RPM').alias('Cooking_RPM'),max('PH').alias('Cooking_PH'),max('Temp').alias('Cooking_Temp'))\
    .select('VatNo','Cooking_Start_Time','Cooking_Stop_Time','Cooking_RPM','Cooking_PH','Cooking_Temp','Report_VAT_Seq','order time','date_id').filter(col('VatNo').isNotNull()).orderBy('Cooking_Start_Time','Report_VAT_Seq')

In [215]:
df_cooking_testing2 = df_cooking_testing1.orderBy('Cooking_Start_Time','Report_VAT_Seq').select('VatNo','Cooking_Start_Time','Report_VAT_Seq','Cooking_RPM','Cooking_PH','Cooking_Temp','Cooking_Stop_Time','order time')\
.withColumn("Date_Hour", date_format(col("order time"), "yyyy-MM-dd HH")).withColumn("Stop_Timestamp", unix_timestamp(concat_ws(" ", date_format(col("Date_Hour"), "yyyy-MM-dd"), col("Cooking_Start_Time")), "yyyy-MM-dd HH:mm:ss") )

# df_cooking_testing2.orderBy('order time').display()

In [140]:
df_cooking_testing2.orderBy('order time').show(df_cooking_testing2.count(),truncate=False)

df_cooking_testing2.count()

25/02/25 14:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+------------------+--------------+-----------+----------+------------+-----------------+--------------------------+-------------+--------------+
|VatNo|Cooking_Start_Time|Report_VAT_Seq|Cooking_RPM|Cooking_PH|Cooking_Temp|Cooking_Stop_Time|order time                |Date_Hour    |Stop_Timestamp|
+-----+------------------+--------------+-----------+----------+------------+-----------------+--------------------------+-------------+--------------+
|1    |06:28:23          |2             |5.0        |0.0       |31.5        |06:29:56         |2024-10-18 06:01:01.284471|2024-10-18 06|1729213103    |
|1    |07:20:15          |3             |4.0        |0.0       |34.6        |07:35:29         |2024-10-18 07:01:01.289936|2024-10-18 07|1729216215    |
|2    |07:01:58          |2             |5.0        |0.0       |31.6        |07:01:58         |2024-10-18 07:01:01.289936|2024-10-18 07|1729215118    |
|2    |07:01:59          |3             |5.0        |0.0       |31.6        |07:02:50   

25/02/25 14:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


94

In [216]:

df_cook_join =  df_cooking_testing2.alias('df1').join(df_join_cooking_df.alias('df2'),
        (col("df1.VatNo") == col("df2.VatNo")) &
        ((col("df1.Date_Hour") == (col("df2.Date_Hour") - 1)) | (col("df1.Date_Hour") == col("df2.Date_Hour"))  | 
            (date_format(col("df1.Cooking_Stop_Time"), "HH:mm") == date_format(col("df2.Cooking_Start_Time"), "HH:mm"))
        ),
# ((col('df1.Date_Hour') == col('df2.Date_Hour')) | (col('df1.Date_Hour') == expr("date_sub(df2.Date_Hour, 1)"))) ,
how='left').withColumn('differe',col('df2.Start_Timestamp') - col('df1.Stop_Timestamp') ).select('df1.VatNo','df1.Cooking_Start_Time','df2.Cooking_Stop_Time','df1.Report_VAT_Seq','df1.Cooking_RPM','df1.Cooking_PH','df1.Cooking_Temp',
greatest(col("df1.Cooking_Temp"), col("df2.max temp")).alias("Cooking_Temp").alias("Cooking Temp"),'order time','differe')\
    .filter( (col('df2.Start_Timestamp') - col('df1.Stop_Timestamp') > 60) & (col('df2.Start_Timestamp') - col('df1.Stop_Timestamp') <= 3000))\
    .withColumn('row_num',row_number().over(Window.partitionBy(col('VatNo'),col('Cooking_Stop_Time')).orderBy(col('Cooking_Start_Time')))).filter(col('row_num') == 1)

# .filter(    (
#         (unix_timestamp(concat_ws(" ", date_format(col("df2.Date_Hour"), "yyyy-MM-dd"), col("df2.Cooking_Stop_Time")), "yyyy-MM-dd HH:mm:ss") - 
#             unix_timestamp(concat_ws(" ", date_format(col("df1.Date_Hour"), "yyyy-MM-dd"), col("df1.Cooking_Stop_Time")), "yyyy-MM-dd HH:mm:ss") > 60)    
#         & (unix_timestamp(concat_ws(" ", date_format(col("df2.Date_Hour"), "yyyy-MM-dd"), col("df2.Cooking_Stop_Time")), "yyyy-MM-dd HH:mm:ss") - 
#             unix_timestamp(concat_ws(" ", date_format(col("df1.Date_Hour"), "yyyy-MM-dd"), col("df1.Cooking_Stop_Time")), "yyyy-MM-dd HH:mm:ss") <= 5700)
#     )
#     # |
#     # (
#     #     (unix_timestamp(concat_ws(" ", current_date(), col("df2.Cooking_Stop_Time"))) - unix_timestamp(concat_ws(" ", current_date(), col("df1.Cooking_Start_Time")),"yyyy-MM-dd HH:mm:ss")> 60)    
#     #     &
#     #     (unix_timestamp(concat_ws(" ", current_date(), col("df2.Cooking_Stop_Time"))) - unix_timestamp(concat_ws(" ", current_date(), col("df1.Cooking_Start_Time")),"yyyy-MM-dd HH:mm:ss") <= 2700)
#     # )
    
    
    
# )

# df_cook_join.orderBy('order time').display()

In [196]:
df_cook_join= df_cook_join.withColumn("Date_Hour2", to_date("order time")).withColumn(
    "Cooking_Start_Timestamp",
    to_timestamp(expr("CONCAT(to_date(`order time`), ' ', Cooking_Start_Time)"), "yyyy-MM-dd HH:mm:ss")
)

In [ ]:
df_cook_join.orderBy('order time').show(df_cook_join.count(),truncate=False)


# df_cook_join.count()

25/02/25 15:35:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:35:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:35:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:35:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:35:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:35:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+------------------+-----------------+--------------+-----------+----------+------------+------------+--------------------------+-------+-------+----------+-----------------------+
|VatNo|Cooking_Start_Time|Cooking_Stop_Time|Report_VAT_Seq|Cooking_RPM|Cooking_PH|Cooking_Temp|Cooking Temp|order time                |differe|row_num|Date_Hour2|Cooking_Start_Timestamp|
+-----+------------------+-----------------+--------------+-----------+----------+------------+------------+--------------------------+-------+-------+----------+-----------------------+
|1    |06:28:23          |06:29:56         |2             |5.0        |0.0       |31.5        |31.5        |2024-10-18 06:01:01.284471|93     |1      |2024-10-18|2024-10-18 06:28:23    |
|1    |07:20:15          |07:54:38         |3             |4.0        |0.0       |34.6        |38.9        |2024-10-18 07:01:01.289936|2063   |1      |2024-10-18|2024-10-18 07:20:15    |
|2    |07:54:39          |08:28:08         |4             |4.0   

In [217]:
df_inter_cutt_join.withColumn('cona_stra',
                        to_timestamp(expr("CONCAT(date_format(`df1`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df1`.`Coagulation_Start_Time` AS STRING))")
                    )).orderBy('order time').show(df_cook_join.count(),truncate=False)

25/02/25 15:39:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:39:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+----------------------+---------------------+--------------------------+------------------+-----------+-------------------------------+------------------------+-------------------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|order time                |date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|Coagulation_Start_Time|Coagulation_Stop_Time|coagt order time          |Cutting_Start_Time|Cutting_RPM|Intermittent_Cutting_Start_Time|Intermittent_Cutting_RPM|cona_stra          |
+-----------------------+-----+--------------+--------+-----------+---

In [219]:
windowSpec_cooking = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(col('Cooking_Start_Time').desc(),abs(unix_timestamp(col("df2.Cooking_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss")))

# df_cook_all_join =  df_inter_cutt_join.alias('df1').join(df_cook_join.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& (abs(unix_timestamp(col("df2.Cooking_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Coagulation_Stop_Time"), "HH:mm:ss")) <= 1800),'left').select('df1.*','df2.*')\
# .withColumn("row_num", row_number().over(windowSpec_cooking)).filter(col("row_num") == 1).drop("row_num").select('df1.*','df2.Cooking_Start_Time','df2.Cooking_Stop_Time','df2.Cooking_RPM','df2.Cooking Temp')
# .filter(
#     (unix_timestamp(col("Cooking_Stop_Time"),"HH:mm:ss") - unix_timestamp(col("Cooking_Start_Time"),"HH:mm:ss")) >= 1800  # 30 min = 1800 sec
# )
df_cook_all_join =  df_inter_cutt_join.alias('df1').join(df_cook_join.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') ) & 
                (abs(
                    unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`df2`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df2`.`Cooking_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ) - 
                    unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`df1`.`coagt order time`, 'yyyy-MM-dd'), ' ', CAST(`df1`.`Coagulation_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    )
                ) <= 7200  # 30 minutes
            ),'left')\
                    .select('df1.*','df2.*')\
.withColumn("row_num", row_number().over(windowSpec_cooking)).filter(col("row_num") == 1).drop("row_num").select('df1.*','df2.Cooking_Start_Time','df2.Cooking_Stop_Time','df2.Cooking_RPM','df2.Cooking Temp')

# df_cook_all_join.orderBy('order time').display()

df_cook_all_join.orderBy('order time').show(df_cook_all_join.count(),truncate=False)

25/02/25 15:40:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:40:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:40:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:40:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:40:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:40:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+----------------------+---------------------+--------------------------+------------------+-----------+-------------------------------+------------------------+------------------+-----------------+-----------+------------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|order time                |date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|Coagulation_Start_Time|Coagulation_Stop_Time|coagt order time          |Cutting_Start_Time|Cutting_RPM|Intermittent_Cutting_Start_Time|Intermittent_Cutting_RPM|Cooking_Start_Time|Cooking_Stop_Time|Cooking_RPM|

In [220]:
emptying = df.withColumn('VatNo',
                            when((col('Report_VAT1_EmptT') == 1) ,1)
                           .when((col('Report_VAT2_EmptT') == 1) ,2)
                           .when((col('Report_VAT3_EmptT') == 1 ) ,3)
                           .when((col('Report_VAT4_EmptT') == 1)  ,4)
                           .when((col('Report_VAT5_EmptT') == 1)  ,5)
                           .when((col('Report_VAT6_EmptT') == 1)  ,6)
                           .when((col('Report_VAT7_EmptT') == 1)  ,7))\
                .withColumn("PH",when((col('Report_VAT1_EmptT') == 1),col('Report_VAT1_PH') )
                           .when((col('Report_VAT2_EmptT') == 1) ,col('Report_VAT2_PH'))
                           .when((col('Report_VAT3_EmptT') == 1 ) ,col('Report_VAT3_PH'))
                           .when((col('Report_VAT4_EmptT') == 1) ,col('Report_VAT4_PH'))
                           .when((col('Report_VAT5_EmptT') == 1) ,col('Report_VAT5_PH'))
                           .when((col('Report_VAT6_EmptT') == 1) ,col('Report_VAT6_PH'))
                           .when((col('Report_VAT7_EmptT') == 1) ,col('Report_VAT7_PH')))

emptying_group = emptying.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Emptyping_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Emptyping_Stop_Time'),min('_start').alias('order time'),max('PH').alias('Empty_PH'))\
    .select('VatNo','Emptyping_Start_Time','Emptyping_Stop_Time','Report_VAT_Seq','order time','Empty_PH').filter(col('VatNo').isNotNull()).orderBy('Emptyping_Start_Time','Report_VAT_Seq').withColumn("stop_time_unix", unix_timestamp(col("Emptyping_Stop_Time"), "HH:mm:ss")).withColumn("Date_Hour", date_format(col("order time"), "yyyy-MM-dd HH"))

In [221]:
emptying_group2 = emptying_group.alias('df1').join(
    emptying_group.alias('df2'),
    (col('df1.VatNo') == col('df2.VatNo')) & 
    (date_format(col("df1.Emptyping_Stop_Time"), "HH:mm") == date_format(col("df2.Emptyping_Start_Time"), "HH:mm")),
    'left'
).withColumn('Emptyping Stop Time',
                        when(col('df2.Emptyping_Stop_Time').isNotNull(),col('df2.Emptyping_Stop_Time'))
                        .otherwise(col('df1.Emptyping_Stop_Time')) 
).select('df1.VatNo','df1.Emptyping_Start_Time','df1.Report_VAT_Seq',col('Emptyping Stop Time').alias('Emptyping_Stop_Time'),'df1.order time','df1.Empty_PH')\
.withColumn("row_number", dense_rank().over(Window.orderBy('Emptyping_Start_Time').partitionBy("VatNo",'Emptyping_Stop_Time')))\
.filter(col('row_number') == 1).drop('row_number')\
.withColumn("Start_Time", first("Emptyping_Start_Time").over(Window.partitionBy("VatNo").orderBy("Report_VAT_Seq")))\
              .withColumn("End_Time", last("Emptyping_Stop_Time").over(Window.partitionBy("VatNo").orderBy("Report_VAT_Seq")))
            #   .select("VatNo", "Start_Time", "End_Time", "Report_VAT_Seq")

In [147]:
emptying_group2.orderBy('df1.order time','df1.Report_VAT_Seq').show(emptying_group2.count(),truncate=False)
emptying_group2.count()

25/02/25 14:54:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----+--------------------+--------------+-------------------+--------------------------+--------+----------+--------+
|VatNo|Emptyping_Start_Time|Report_VAT_Seq|Emptyping_Stop_Time|order time                |Empty_PH|Start_Time|End_Time|
+-----+--------------------+--------------+-------------------+--------------------------+--------+----------+--------+
|1    |08:25:53            |5             |09:04:04           |2024-10-18 08:01:01.286304|6.45    |04:01:42  |09:04:04|
|2    |09:04:05            |6             |09:41:43           |2024-10-18 09:01:01.287965|6.44    |04:34:34  |09:41:43|
|3    |09:41:44            |7             |10:16:46           |2024-10-18 09:31:01.293729|0.0     |09:41:44  |10:16:46|
|4    |10:16:47            |8             |10:48:35           |2024-10-18 10:01:01.28685 |0.0     |10:16:47  |10:48:35|
|5    |10:48:36            |9             |11:16:35           |2024-10-18 10:31:01.292736|6.47    |10:48:36  |11:16:35|
|6    |11:16:36            |10          

25/02/25 14:54:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 14:54:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

40

In [148]:
# df_Emptying  =   df.withColumn('VatNo',
#                             when((col('Report_VAT1_EmptT') == 1) & ((col('Report_VAT5_Filling') == 1) | ((col('Report_VAT_Seq') == 0)  &  (col('Report_VAT7_EmptT') == 1))),1)
#                            .when((col('Report_VAT2_EmptT') == 1) & ((col('Report_VAT6_Filling') == 1) | ((col('Report_VAT_Seq') == 0) &  (col('Report_VAT1_EmptT') == 1)  )),2)
#                            .when((col('Report_VAT3_EmptT') == 1 ) & ((col('Report_VAT7_Filling') == 1) | ((col('Report_VAT_Seq') == 0) &  (col('Report_VAT2_EmptT') == 1) )),3)
#                            .when((col('Report_VAT4_EmptT') == 1) & ((col('Report_VAT1_Filling') == 1) | ((col('Report_VAT_Seq') == 0) &  (col('Report_VAT3_EmptT') == 1) )) ,4)
#                            .when((col('Report_VAT5_EmptT') == 1) & ((col('Report_VAT2_Filling') == 1) | ((col('Report_VAT_Seq') == 0) &  (col('Report_VAT4_EmptT') == 1) )) ,5)
#                            .when((col('Report_VAT6_EmptT') == 1) & ((col('Report_VAT3_Filling') == 1) | ((col('Report_VAT_Seq') == 0) &  (col('Report_VAT5_EmptT') == 1) )) ,6)
#                            .when((col('Report_VAT7_EmptT') == 1) & ((col('Report_VAT4_Filling') == 1) | ((col('Report_VAT_Seq') == 0) &  (col('Report_VAT6_EmptT') == 1) )) ,7))\
#                 .withColumn("PH",when((col('Report_VAT1_EmptT') == 1) & ((col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),col('Report_VAT1_PH'))
#                            .when((col('Report_VAT2_EmptT') == 1) & ((col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0)),col('Report_VAT2_PH'))
#                            .when((col('Report_VAT3_EmptT') == 1 ) & ((col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0)),col('Report_VAT3_PH'))
#                            .when((col('Report_VAT4_EmptT') == 1) & ((col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0)),col('Report_VAT4_PH'))
#                            .when((col('Report_VAT5_EmptT') == 1) & ((col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0)),col('Report_VAT5_PH'))
#                            .when((col('Report_VAT6_EmptT') == 1) & ((col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0)),col('Report_VAT6_PH'))
#                            .when((col('Report_VAT7_EmptT') == 1) & ((col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0)),col('Report_VAT7_PH')))

In [149]:
# df_Emptying_group = df_Emptying.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Emptyping_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Emptyping_Stop_Time'),min('_start').alias('order time'),max('PH').alias('Empty_PH')).select('VatNo','Emptyping_Start_Time','Emptyping_Stop_Time','Empty_PH','Report_VAT_Seq','order time').filter(col('VatNo').isNotNull()).orderBy('Emptyping_Start_Time','Report_VAT_Seq').withColumn("stop_time_unix", unix_timestamp(col("Emptyping_Stop_Time"), "HH:mm:ss")).withColumn("Date_Hour", date_format(col("order time"), "yyyy-MM-dd HH"))

In [150]:
# df_Emptying_group.orderBy('order time').show(df_Emptying_group.count(),truncate=False)

# df_Emptying_group.count()

In [151]:
# df_Emptying_last =  df.withColumn('VatNo',
#                            when((col('Report_VAT1_EmptT') == 1) & ((col('Report_VAT6_Filling') == 1) | (col('Report_VAT_Seq') == 0) ),1)
#                            .when((col('Report_VAT2_EmptT') == 1) & ((col('Report_VAT7_Filling') == 1) | (col('Report_VAT_Seq') == 0)),2)
#                            .when((col('Report_VAT3_EmptT') == 1 ) & ((col('Report_VAT1_Filling') == 1) | (col('Report_VAT_Seq') == 0)),3)
#                            .when((col('Report_VAT4_EmptT') == 1) & ((col('Report_VAT2_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,4)
#                            .when((col('Report_VAT5_EmptT') == 1) & ((col('Report_VAT3_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,5)
#                            .when((col('Report_VAT6_EmptT') == 1) & ((col('Report_VAT4_Filling') == 1) | (col('Report_VAT_Seq') == 0)) ,6)
#                            .when((col('Report_VAT7_EmptT') == 1) & ((col('Report_VAT5_Filling') == 1) | (col('Report_VAT_Seq') == 0)), 7))

In [152]:
# df_Emptying_last2 =  df_Emptying_last.groupBy('VatNo','Report_VAT_Seq','date_id').agg(date_format(min(col("_time")).cast("timestamp"),"HH:mm:ss").alias('Emptyping_Start_Time'),date_format(max(col("_time")).cast("timestamp"), "HH:mm:ss").alias('Emptyping_Stop_Time'),min('_start').alias('order time')).select('VatNo','Emptyping_Start_Time','Emptyping_Stop_Time','Report_VAT_Seq','order time').filter(col('VatNo').isNotNull()).orderBy('Emptyping_Start_Time','Report_VAT_Seq').withColumn("start_time_unix", unix_timestamp(col("Emptyping_Start_Time"), "HH:mm:ss")).withColumn("Date_Hour", date_format(col("order time"), "yyyy-MM-dd HH"))

In [153]:
# df_Emptying_last2.orderBy('order time').show(df_Emptying_last2.count(),truncate=False)

In [154]:
# df_Emptying_join = df_Emptying_group.alias('df1').join(df_Emptying_last2.alias('df2'),(col('df1.VatNo') == col('df2.VatNo')) &
#             (
#               abs(
#                   unix_timestamp(col("df2.Emptyping_Start_Time"), "HH:mm:ss") - 
#                   unix_timestamp(col("df1.Emptyping_Stop_Time"), "HH:mm:ss")
#               ) <= 300  # Less than 5 minutes
#           ), 'left')\
#     .select('df1.VatNo','df1.Emptyping_Start_Time','df2.Emptyping_Stop_Time','df1.Empty_PH','df1.order time').withColumn("row_num", row_number().over(Window.partitionBy('df1.VatNo', 'df1.Emptyping_Start_Time').orderBy(col('df2.Emptyping_Stop_Time').desc())))\
#         # .filter(col("row_num") == 1).drop("row_num")

In [155]:
# df_Emptying_join.orderBy('order time').show(df_Emptying_join.count(),truncate=False)

# df_Emptying_join.count()
# windowSpec_empty = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(col('Emptyping_Start_Time').desc(),abs(unix_timestamp(col("df2.Emptyping_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")))

# df_Emptying_join_all =  df_cook_all_join.alias('df1').join(emptying_group2.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& 
#         (abs(unix_timestamp(col("df2.Emptyping_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")).between(6200, 8000)),'left').select('df1.*','df2.Emptyping_Start_Time','df2.Emptyping_Stop_Time','df2.Empty_PH').withColumn("row_num", row_number().over(windowSpec_empty)).filter(col("row_num") == 1).drop("row_num").drop('df2.Report_VAT_Seq')



In [222]:
windowSpec_empty = Window.partitionBy("df1.VatNo", "df1.Filling_end_time").orderBy(col('Emptyping_Start_Time').desc(),abs(unix_timestamp(col("df2.Emptyping_Start_Time"), "HH:mm:ss") - unix_timestamp(col("df1.Filling_end_time"), "HH:mm:ss")))

df_Emptying_join_all =  df_cook_all_join.alias('df1').join(emptying_group2.alias('df2'),(col('df1.VatNo') == col('df2.VatNo') )& 
        (abs(
            unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`df2`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df2`.`Emptyping_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ) - 
                    unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`df1`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df1`.`Filling_end_time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    )).between(6200, 8000)),'left').select('df1.*','df2.Emptyping_Start_Time','df2.Emptyping_Stop_Time','df2.Empty_PH',col('df2.order time').alias('empty order time')).withColumn("row_num", row_number().over(windowSpec_empty)).filter(col("row_num") == 1).drop("row_num").drop('df2.Report_VAT_Seq')

In [223]:
df_Emptying_join_all.orderBy('order time').show(df_Emptying_join_all.count(),truncate=False)

25/02/25 15:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:45:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+-----------+------------------+--------------------+-------------------+--------------------------+-------+----------------+-----------------+---------------------+----------------+----------+--------------------+---------------+----------------------+---------------------+--------------------------+------------------+-----------+-------------------------------+------------------------+------------------+-----------------+-----------+------------+--------------------+-------------------+--------+--------------------------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling_RPM|Filling Start Time|Milk Quantity in Vat|Temp of milk in Vat|order time                |date_id|Filling_end_time|Milk_Quantity_Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Addition_Time|Rennet_Quantity|Coagulation_Start_Time|Coagulation_Stop_Time|coagt order time          |Cutting_Start_Time|Cutting_RPM|Intermittent_Cutting_Start_Ti

In [ ]:
# # vat_final_df = df_Emptying_join_all.withColumn("Residence Time in Vat", abs(unix_timestamp(col("Emptyping_Start_Time"), "HH:mm:ss") - unix_timestamp(col("Filling_end_time"), "HH:mm:ss"))).withColumn("Residence Time in Vat", from_unixtime(col("Residence Time in Vat"), "HH:mm:ss")).withColumn("Total Process time in VAT", abs(unix_timestamp(col("Emptyping_Stop_Time"), "HH:mm:ss") - unix_timestamp(col("Filling Start Time"), "HH:mm:ss"))).withColumn("Total Process time in VAT", from_unixtime(col("Total Process time in VAT"), "HH:mm:ss"))\
# #     .select('Date','VatNo','Report_VAT_Seq','RecipeNo','Filling Start Time','Filling_end_time','Filling_RPM','Milk Quantity in Vat','Temp of milk in Vat','Culture_Addition_Time','Culture_Quantity','Culture_ph','Rennet_Quantity','Rennet_Addition_Time','Coagulation_Start_Time','Coagulation_Stop_Time','Cutting_Start_Time','Cutting_RPM','Intermittent_Cutting_Start_Time','Intermittent_Cutting_RPM','Cooking_Start_Time','Cooking_Stop_Time','Cooking_RPM','Cooking Temp','Emptyping_Start_Time','Emptyping_Stop_Time','Empty_PH','Residence Time in Vat','Total Process time in VAT')


# abs(
#             unix_timestamp(
#                         to_timestamp(expr("CONCAT(date_format(`df2`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df2`.`Emptyping_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
#                     ) - 
#                     unix_timestamp(
#                         to_timestamp(expr("CONCAT(date_format(`df1`.`order time`, 'yyyy-MM-dd'), ' ', CAST(`df1`.`Filling_end_time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
#                     )



SyntaxError: unexpected EOF while parsing (196645497.py, line 12)

In [224]:
vat_final_df = df_Emptying_join_all.withColumn(
    "Adjusted_empty_order_date",
    when(
        (col("Emptyping_Stop_Time").isNotNull()) &  
        (hour(col("Emptyping_Stop_Time")) >= 0) & (hour(col("Emptyping_Stop_Time")) < 2) &  
        (to_date(col("empty order time")) == to_date(col("empty order time"))) &  # Ensures valid date
        (to_date(col("empty order time")) == to_date(col("empty order time")) + 1),  # Prevents duplicate addition
        date_add(to_date(col("empty order time")), 1)  # Add 1 day only if not already added
    ).otherwise(to_date(col("empty order time")))  # Keep the same date otherwise
)\
  .withColumn("Residence Time in Vat", 
                abs( unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`empty order time`, 'yyyy-MM-dd'), ' ', CAST(`Emptyping_Start_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ) - 
                      unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`order time`, 'yyyy-MM-dd'), ' ', CAST(`Filling_end_time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ))
               ) \
    .withColumn("Residence Time in Vat",
                concat(
                    lpad((col("Residence Time in Vat") / 3600).cast("int"), 2, '0'), 
                    lit(':'), 
                    lpad(((col("Residence Time in Vat") % 3600) / 60).cast("int"), 2, '0'), 
                    lit(':'), 
                    lpad((col("Residence Time in Vat") % 60).cast("int"), 2, '0')
                ))   \
    .withColumn("Total Process Time in VAT", 
                abs( unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`Adjusted_empty_order_date`, 'yyyy-MM-dd'), ' ', CAST(`Emptyping_Stop_Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ) - 
                      unix_timestamp(
                        to_timestamp(expr("CONCAT(date_format(`order time`, 'yyyy-MM-dd'), ' ', CAST(`Filling Start Time` AS STRING))"), "yyyy-MM-dd HH:mm:ss")
                    ))
               ) \
    .withColumn("Total Process time in VAT",
                concat(
                    lpad((col("Total Process time in VAT") / 3600).cast("int"), 2, '0'), 
                    lit(':'), 
                    lpad(((col("Total Process time in VAT") % 3600) / 60).cast("int"), 2, '0'), 
                    lit(':'), 
                    lpad((col("Total Process time in VAT") % 60).cast("int"), 2, '0')
                ))\
    .select('Date','VatNo','Report_VAT_Seq','RecipeNo','Filling Start Time','Filling_end_time','Filling_RPM','Milk Quantity in Vat','Temp of milk in Vat','Culture_Addition_Time','Culture_Quantity','Culture_ph','Rennet_Quantity','Rennet_Addition_Time','Coagulation_Start_Time','Coagulation_Stop_Time','Cutting_Start_Time','Cutting_RPM','Intermittent_Cutting_Start_Time','Intermittent_Cutting_RPM','Cooking_Start_Time','Cooking_Stop_Time','Cooking_RPM','Cooking Temp','Emptyping_Start_Time','Emptyping_Stop_Time','Empty_PH','Residence Time in Vat','Total Process time in VAT')

25/02/25 15:45:57 WARN Column: Constructing trivially true equals predicate, 'to_date('empty order time, None, None, false) = to_date('empty order time, None, None, false)'. Perhaps you need to use aliases.


In [225]:
vat_final_df.orderBy('Date').show(vat_final_df.count(),truncate=False)

# vat_final_df.count()

25/02/25 15:46:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:02 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

+-----------------------+-----+--------------+--------+------------------+----------------+-----------+--------------------+-------------------+---------------------+----------------+----------+---------------+--------------------+----------------------+---------------------+------------------+-----------+-------------------------------+------------------------+------------------+-----------------+-----------+------------+--------------------+-------------------+--------+---------------------+-------------------------+
|Date                   |VatNo|Report_VAT_Seq|RecipeNo|Filling Start Time|Filling_end_time|Filling_RPM|Milk Quantity in Vat|Temp of milk in Vat|Culture_Addition_Time|Culture_Quantity|Culture_ph|Rennet_Quantity|Rennet_Addition_Time|Coagulation_Start_Time|Coagulation_Stop_Time|Cutting_Start_Time|Cutting_RPM|Intermittent_Cutting_Start_Time|Intermittent_Cutting_RPM|Cooking_Start_Time|Cooking_Stop_Time|Cooking_RPM|Cooking Temp|Emptyping_Start_Time|Emptyping_Stop_Time|Empty_PH

In [ ]:
blob_path = '/Users/sushantshinde/workspaces/trvi/output'

In [226]:
vat_final_df.repartition(1).orderBy('Report_VAT_Seq') \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(blob_path)

25/02/25 15:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 15:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/25 1

In [227]:
import shutil
import glob

output_folder = "/Users/sushantshinde/workspaces/trvi/output"
csv_files = glob.glob(f"{output_folder}/part-*.csv")

if csv_files:
    shutil.move(csv_files[0], "/Users/sushantshinde/workspaces/trvi/output_csv/18oct.csv")